Bramki kwantowe realizowane są w modelu bramkowym przez `operatory unitarne` reprezentowane przez macierze.

$$
U U^{\dagger} = U^{\dagger} U = I 
$$

Kazda macierz unitarna moze być przedstawiona jako: 

$$ 
U(H,t) = e^{-i H t}
$$
gdzie $H$ to macierz Hermitowska ($H=H^{\dagger}$)

W ogólności, implementacja obwodu kwantowego, który dokładnie realizuje macierz unitarną dla zadanego 
Hamiltonianiu jest bardzo trudnym zadaniem. Hamiltonian taki zazwyczaj składa się z sumy wielu niekomutujących części.  

$$ 
H = H_1 + H_2 + \dots + H_n
$$ 

Mozemy wykorzystać wzór  `Trotter'a-Suzuki` który przybliza dowolną sumę macierzy 
$$ 
e^{A + B} \approx \left( e^{A/n} e^{B/n} \right)^n
$$ 
 
dlatego dla  $H=\sum_k H_k$ otrzymujemy
 $$ U(H,t,n) = \prod_{j=1}^n \prod_k e^{-i H_k t/n} $$

# QUBO 


Kombinatoryczne problemy opytmalizacyjne realizowane są na wielu płaszczyznach naukowych i aplikacyjnych:

- logistyka,
- planowanie, 
- optymalizacja portfolio,
- ...

Combinatorial optimization problems are problems involving a large number of yes/no decisions with each set of decisions yielding a corresponding objective function value, like a cost or profit value.

Because of the combinatorial explosion of the solution space with the number of variables, finding good solutions is extremely difficult.

The QUBO model unifies a rich variety of NP-hard combinatorial optimization problems:

- Quadratic Assignment Problems 
- Capital Budgeting Problems
- Task allocation Problems
- Maximum--Cut Problems

QUBO objective function:

$$
F(q) = \sum_a v_a q_a + \sum_{a < b} \omega_{a b} q_a q_b 
$$
gdzie $q_a \in \{0,1\}$, $v_a$ oraz $\omega_a$ to rzeczywiste współczynniki dla liniowej i kwadratowej części.

The QUBO objective function is NP-hard in nature.

Wprowadzmy zamianę zmiennych: 
$$ 
z_a = 2q_a-1 
$$ 
gdzie $z \in {-1,1}$

$$ 
F(z) = \sum_a h_a z_a + \sum_{a < b} J_{a b} z_a z_b 
$$ 



One popular method of encoding an optimization problem to be solved using QAOA, is to first formulate the problem as an `Ising Objective function`. The Ising model is a popular statistical mechanics model, associated primarily with ferromagnetism. Because it has been shown to be NP-Complete in nature, the objective function associated with it can be used to represent hard problems.

### Max-Cut

**Max-Cut** is an NP-complete problem, with applications in clustering, network science, and statistical physics. 

Given a graph $G(V,E)$, we seek partition of $V$ into two subsets with maximum cut. 

In short, we have to color every node either blue or red and we score a point whenever an edge connects two nodes with different colors. We then would like to find the solution with the highest score. 


<img src="img\qaoa_maxcut.png" height="300" >

Again, the problem in this specific graph coloring problem is that there are $2^N$ possible solutions for $N$ nodes (an exponential explosion in possibilities), making it impossible to enumerate all possible candidates for relevant system sizes.

The solution of Max-Cut, even if approximate, has practical application in machine scheduling, image recognition or for the layout of electronic circuits.

We can encode the `Maximum Cut` problem as a minimization problem of an Ising Hamiltonian, where the (classical) cost function reads:
$$ H_C = \sum_{a < b} J_{a b} z_a z_b $$ 

Ising matrix $J$ encoding the weights of the edges.

In short, the cost Hamiltonian assigns a number to every bitstring $z=(z_1,z_2,\dots,z_n)$
, and we would like to find the lowest number possible. This will be the optimal assignment and solution to our problem.

It is important to note here that we still don’t know if quantum computing can help solve NP-Complete problems efficiently. 
Our hope for quantum algorithms, at the very least, is to be able to compete with classical heuristics when it comes to certain classes of hard problems.


The quantum Ising Hamiltonian, which naturally maps the Ising objective to qubits:

$$\hat{C} = \sum_{a < b} J_{a b} \hat{\sigma}_a^z \hat{\sigma}_b^z $$ 
which can be written as a matrix of size $(2^N, 2^N)$ with diagonal elements only corresponding to all possible classical values for the cost function $\hat{C}$.

Because qubits are 2-dim vectors $\sigma_a^z$ correspond to 2x2 matrix with two eigenvalue $\{1,-1\}$ and two eigenvectors $$|0> = \begin{pmatrix} 1 \\ 0 \end{pmatrix}$$
$$|1> = \begin{pmatrix} 0 \\ 1 \end{pmatrix}$$

So

$$ \sigma^z = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix}$$

and 

$$ \sigma^z_a = \left( \otimes_{i=1}^{a-1} I \right) \otimes \left( \sigma^z \right) \otimes \left(\otimes_{i=a+1}^{n} I \right)$$


The other type of Hamiltonian in the QAOA process is a summation of individual Pauli X operators for each qubit involved in the process, which intuitively represents $$\hat{B}=\sum_a \sigma^x_a$$ transverse field in the Ising model
$$ \sigma^x = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}$$

$$ \sigma^x_a = \left( \otimes_{i=1}^{a-1} I \right) \otimes \left( \sigma^x \right) \otimes \left(\otimes_{i=a+1}^{n} I \right)$$

The ground state of this Hamiltonian corresponds to the optimal solution of the classical combinatorial problem.

Finding this ground state is generically hard.

### Realizacja kodu w pythonie

Functional programming breaks down an application into a set of functions. 
Ideally, functions only take inputs and produce outputs and have no internal state that affects the output produced for a given input.

In that sense, the `QAOA algorithm` is a function that solves a `problem` by `optimize`ing a set of `params`. In other words, we aim to find the best values for these params.

To decide which params are best, we assess them based on the result we obtain from `compute`ing a (quantum) circuit that uses these params to encode the `problem` (problem_circuit) and its solution (ansatz_circuit).

This is what Qiskit’s description refers to as a variational algorithm. 

It uses a classical optimization algorithm that makes queries to a quantum computer.


In [ ]:
def qaoa(problem, optimize, assess, compute,
  to_circuit, problem_circuit, ansatz_circuit):

    return optimize(
        lambda params: assess(problem, compute(to_circuit(problem, params,
              problem_circuit, ansatz_circuit)))
    )

In [ ]:
from qiskit import Aer, execute

def compute(circuit):
    return execute(circuit, 
                   Aer.get_backend('qasm_simulator'), 
                   shots=1000).result().get_counts()

In [ ]:
from qiskit import QuantumCircuit

def to_circuit(problem, params, problem_circuit, ansatz_circuit):
    
    cnt_qubits = problem.size
    
    qc_qaoa = QuantumCircuit(cnt_qubits)

    # initial_state
    qc_qaoa.h(range(cnt_qubits))
    
    # append problem circuit
    qc_qaoa.append(problem_circuit(problem, params[0]), range(cnt_qubits))
    
    # append ansatz circuit
    qc_qaoa.append(ansatz_circuit(problem, params[1]), range(cnt_qubits))
    qc_qaoa.measure_all()
    
    return qc_qaoa